In [1]:
from init import *

In [2]:
spot=1
strike=1.1
volatility=0.3

model = Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 100000
model.RandomSeed = 123

model.Add(IndependentBrownianMotion())

underlying_process = GeometricalBrownianMotion(
    start = spot,
    drift = 0.0,
    diffusion = volatility,
    title = 'Underlying'
)
model.Add(underlying_process)

call_option_process = Product_Option(
    underlying = underlying_process.Number(),
    strike = strike,
    call_put = Product_Option.Call,
    title = f'Call, K={strike}'
)
model.Add(call_option_process)

put_option_process = Product_Option(
    underlying = underlying_process.Number(),
    strike = strike,
    call_put = Product_Option.Put,
    title = f'Put, K={strike}'
)
model.Add(put_option_process)

In [3]:
model.evaluations.append(
    EvaluationPoint(state=0,time=2)
)

model.histograms.append(
    Histogram(
        evaluation_point=0,
        x = HistogramAxis(call_option_process.Number(),20,0,2),
        y = HistogramAxis( put_option_process.Number(),20,0,2)
    )
)

model.json()

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "GeometricalBrownianMotion", "start": 1, "args": [0.0, 0.3]}, {"name": "Product_Option", "start": null, "refs": [0], "args": [1.1, 0]}, {"name": "Product_Option", "start": null, "refs": [0], "args": [1.1, 1]}], "evaluations": [{"state": 0, "time": 2}], "RandomSeed": 123, "histograms": [{"evaluation_point": 0, "x": {"state": 1, "bins": 20, "min": 0, "max": 2}, "y": {"state": 2, "bins": 20, "min": 0, "max": 2}}]}'

In [4]:
reply = requests.post(f'{server}/model',model.json())
results = EvaluationResults(reply.json(),model)
results

3 states with 1 evaluations

In [5]:
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,GeometricalBrownianMotion,Underlying,0,2.0,1000,100000,0.996995,0.001400,0.442721,1.400737
1,Product_Option,"Call, K=1.1",1,2.0,1000,100000,0.129407,0.000912,0.288445,3.397551
2,Product_Option,"Put, K=1.1",2,2.0,1000,100000,0.232412,0.000726,0.229454,0.567413
